In [3]:
import pickle
import pandas as pd
from os import listdir
from os.path import isfile, join, isdir

import torch
import torch.multiprocessing

from helpers.tools import *
from helpers.tools import lagged_batch_corr
from model.model_wrapper import Architecture_PL


In [4]:
# Complete summary of all parameter searches that were performed. 

# Naming conventions changed towards to end of the paper: 
#SL: base
#ML: big_base_mirror
#SNL: poly
#MNL: big_poly
#LNL: nl
#XLNL: big_nl

In [5]:
def load_res_folder(mypath = "experimental_results/synthetic_data_1_grid_results/std/"):

    onlyfiles = [mypath + f for f in listdir(mypath) if isfile(join(mypath, f))]
    onlyfiles = sorted([x for x in onlyfiles if "summary" in x])
    res = pd.concat([pd.read_csv(x, index_col=0) for x in onlyfiles], axis=1).T
    return  res

# Experiment 1

In [71]:
a = load_res_folder("experimental_results/synthetic_data_1_grid_results/run_1/")
b = load_res_folder("experimental_results/synthetic_data_1_grid_results/run_2/")

In [74]:
res = load_res_folder("experimental_results/synthetic_data_1_grid_results/std/")
res.loc[res["model"].isnull(), "model"] = "small.yaml"
# Provide some summary statistics over the runs.
res = res[["model.model_type", "data.ds_name",'Test1_Auroc', 'Test2_AUROC']]
res["Test1_Auroc"] = res["Test1_Auroc"].astype(float)
res["Test2_AUROC"] = res["Test2_AUROC"].astype(float)

res.groupby(["data.ds_name", "model.model_type"]).mean()

HP                                Test1_Auroc  Test2_AUROC
data.ds_name    model.model_type                          
base            bidirectional        0.999499     0.999961
                convM                0.999336     0.999894
                mlp                  0.999328     0.999933
                transformer          0.999534     0.999968
                unidirectional       0.999558     0.999948
big_base_mirror bidirectional        0.613235     0.626371
                convM                0.506232     0.513293
                mlp                  0.499207     0.498309
                transformer          0.600757     0.621380
                unidirectional       0.599960     0.621146
big_nl          bidirectional        0.709532     0.748808
                convM                0.503338     0.506151
                mlp                  0.501747     0.498722
                transformer          0.518067     0.517684
                unidirectional       0.659285     0.688653
big_poly        bidirectional        0.547126     0.555322
                convM                0.518390     0.518896
                mlp                  0.520201     0.524884
                transformer          0.537860     0.545076
                unidirectional       0.784930     0.828722
nl              bidirectional        0.916400     0.936746
                convM                0.912467     0.934696
                mlp                  0.905332     0.932553
                transformer          0.915942     0.934492
                unidirectional       0.915071     0.936571
poly            bidirectional        0.932779     0.948389
                convM                0.931417     0.948634
                mlp                  0.930141     0.950117
                transformer          0.934952     0.955290
                unidirectional       0.934503     0.954934

# Experiment 1 - Joint

In [75]:
c = load_res_folder("experimental_results/synthetic_data_1_joint_grid_results/splitted_run_1/")

In [76]:
res = load_res_folder("experimental_results/synthetic_data_1_joint_grid_results/std/")

# Provide some summary statistics over the runs.
res = res[["model.model_type", "model",'Test1_Auroc', 'Test2_AUROC']]
res["Test1_Auroc"] = res["Test1_Auroc"].astype(float)
res["Test2_AUROC"] = res["Test2_AUROC"].astype(float)

res.groupby(["model", "model.model_type"]).mean()

HP                            Test1_Auroc  Test2_AUROC
model       model.model_type                          
big.yaml    bidirectional        0.952198     0.962480
            convM                0.867127     0.885812
            mlp                  0.751937     0.760817
            transformer          0.969769     0.977339
            unidirectional       0.946323     0.959416
medium.yaml bidirectional        0.906445     0.920411
            convM                0.869493     0.887307
            mlp                  0.915602     0.928056
            transformer          0.938883     0.949564
            unidirectional       0.894770     0.906847

# Experiment 2

In [12]:
d = load_res_folder("experimental_results/synthetic_data_2_grid_results/splitted_run_2/")

In [18]:
# We dropped quite a bit of runs initially so we might undereport.
d.groupby(["data.ds_name", "model", "model.model_type"]).count()

HP                                            model.optimizer_lr   
data.ds_name    model       model.model_type                       
big_base_mirror big.yaml    bidirectional                     24  \
                            convM                             24   
                            transformer                       23   
                            unidirectional                    24   
                medium.yaml bidirectional                     24   
...                                                          ...   
scale_up_3      lcm.yaml    bidirectional                     24   
                            convM                             17   
                            mlp                               24   
                            transformer                       10   
                            unidirectional                    24   

HP                                            data.batch_size   
data.ds_name    model       model.model_type                    
big_base_mirror big.yaml    bidirectional                  24  \
                            convM                          24   
                            transformer                    23   
                            unidirectional                 24   
                medium.yaml bidirectional                  24   
...                                                       ...   
scale_up_3      lcm.yaml    bidirectional                  24   
                            convM                          17   
                            mlp                            24   
                            transformer                    10   
                            unidirectional                 24   

HP                                            model.weight_decay   
data.ds_name    model       model.model_type                       
big_base_mirror big.yaml    bidirectional                     24  \
                            convM                             24   
                            transformer                       23   
                            unidirectional                    24   
                medium.yaml bidirectional                     24   
...                                                          ...   
scale_up_3      lcm.yaml    bidirectional                     24   
                            convM                             17   
                            mlp                               24   
                            transformer                       10   
                            unidirectional                    24   

HP                                            model.corr_regularization   
data.ds_name    model       model.model_type                              
big_base_mirror big.yaml    bidirectional                            24  \
                            convM                                    24   
                            transformer                              23   
                            unidirectional                           24   
                medium.yaml bidirectional                            24   
...                                                                 ...   
scale_up_3      lcm.yaml    bidirectional                            24   
                            convM                                    17   
                            mlp                                      24   
                            transformer                              10   
                            unidirectional                           24   

HP                                            corr_input  regression_head   
data.ds_name    model       model.model_type                                
big_base_mirror big.yaml    bidirectional             24               24  \
                            convM                     24               24   
                            transformer               23               23   
                       

In [9]:
res = load_res_folder("experimental_results/synthetic_data_2_grid_results/std/")


In [11]:
res = load_res_folder("experimental_results/synthetic_data_2_grid_results/std/")

# Provide some summary statistics over the runs.
res = res[["data.ds_name","model.model_type", "model",'Test1_Auroc', 'Test2_AUROC']]
res["Test1_Auroc"] = res["Test1_Auroc"].astype(float)
res["Test2_AUROC"] = res["Test2_AUROC"].astype(float)

res.groupby(["data.ds_name", "model", "model.model_type"]).mean()

HP                                            Test1_Auroc  Test2_AUROC
data.ds_name    model       model.model_type                          
big_base_mirror big.yaml    bidirectional        0.768005     0.799255
                            convM                0.500173     0.509324
                            transformer          0.782039     0.837801
                            unidirectional       0.936127     0.983861
                medium.yaml bidirectional        0.665588     0.717873
                            convM                0.505948     0.512882
                            mlp                  0.499003     0.503274
                            transformer          0.613058     0.671812
                            unidirectional       0.640511     0.686944
                small.yaml  bidirectional        0.609612     0.663376
                            convM                0.507054     0.516137
                            mlp                  0.499303     0.500034
                            transformer          0.554291     0.588408
                            unidirectional       0.638393     0.684325
scale_up        big.yaml    bidirectional        0.497698     0.501899
                            convM                0.500272     0.503472
                            transformer          0.499660     0.499479
                            unidirectional       0.501040     0.502786
                medium.yaml bidirectional        0.499616     0.502193
                            convM                0.499371     0.503926
                            mlp                  0.501486     0.502013
                            transformer          0.500681     0.500457
                            unidirectional       0.500426     0.503040
scale_up_2      big.yaml    bidirectional        0.663370     0.712364
                            convM                0.503263     0.512655
                            transformer          0.592004     0.626630
                            unidirectional       0.801570     0.877453
                medium.yaml bidirectional        0.576166     0.606176
                            convM                0.500987     0.505757
                            mlp                  0.501264     0.500298
                            transformer          0.498828     0.500111
                            unidirectional       0.611909     0.644258
                small.yaml  bidirectional        0.503146     0.503853
                            convM                0.500534     0.505084
                            mlp                  0.500084     0.500434
                            transformer          0.499337     0.503212
                            unidirectional       0.501459     0.508311
scale_up_3      big.yaml    bidirectional        0.501505     0.500152
                            convM                0.500696     0.502594
                            transformer          0.500103     0.497545
                            unidirectional       0.499893     0.498777

# Experiment Kuramoto

In [19]:
e = load_res_folder("experimental_results/kuramoto_grid_results/splitted_run_1/")

In [20]:
res = load_res_folder("experimental_results/kuramoto_grid_results/std/")

# Provide some summary statistics over the runs.
res = res[["model.model_type",'Test1_Auroc']]
res["Test1_Auroc"] = res["Test1_Auroc"].astype(float)

res.groupby([ "model.model_type"]).mean()

HP,Test1_Auroc
model.model_type,
bidirectional,0.852495
convM,0.954027
mlp,0.824648
transformer,0.888369
unidirectional,0.905020
